<a href="https://colab.research.google.com/github/oxanaRC/CE888/blob/master/Project3/unit_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# This is a very simple implementation of the UCT Monte Carlo Tree Search algorithm in Python 2.7.
# The function UCT(rootstate, itermax, verbose = False) is towards the bottom of the code.
# It aims to have the clearest and simplest possible code, and for the sake of clarity, the code
# is orders of magnitude less efficient than it could be made, particularly by using a 
# state.GetRandomMove() or state.DoRandomRollout() function.
# 
# Example GameState classes for Nim, OXO and Othello are included to give some idea of how you
# can write your own GameState use UCT in your 2-player game. Change the game to be played in 
# the UCTPlayGame() function at the bottom of the code.
# 
# Written by Peter Cowling, Ed Powley, Daniel Whitehouse (University of York, UK) September 2012.
# 
# Licence is granted to freely use and distribute for any sensible/legal purpose so long as this comment
# remains in any distributed code.
# 
# For more information about Monte Carlo Tree Search check out our web site at www.mcts.ai
#---------------------------------------------------------------------------------------------

In [0]:
from math import *
import random
import numpy as np
import csv
import pandas as pd 

In [0]:
class GameState:
    """ A state of the game, i.e. the game board. These are the only functions which are
        absolutely necessary to implement UCT in any 2-player complete information deterministic 
        zero-sum game, although they can be enhanced and made quicker, for example by using a 
        GetRandomMove() function to generate a random move during rollout.
        By convention the players are numbered 1 and 2.
    """
    def __init__(self):
            self.playerJustMoved = 2 # At the root pretend the player just moved is player 2 - player 1 has the first move

        
    def Clone(self):
        """ Create a deep clone of this game state.
        """
        st = GameState()
        st.playerJustMoved = self.playerJustMoved
        return st

    def DoMove(self, move):
        """ Update a state by carrying out the given move.
            Must update playerJustMoved.
        """
        self.playerJustMoved = 3 - self.playerJustMoved
        
    def GetMoves(self):
        """ Get all possible moves from this state.
        """
    
    def GetResult(self, playerjm):
        """ Get the game result from the viewpoint of playerjm. 
        """

    def __repr__(self):
        """ Don't need this - but good style.
        """
        pass


class OXOState:
    """ A state of the game, i.e. the game board.
        Squares in the board are in this arrangement
        012
        345
        678
        where 0 = empty, 1 = player 1 (X), 2 = player 2 (O)
    """
    def __init__(self):
        self.playerJustMoved = 2 # At the root pretend the player just moved is p2 - p1 has the first move
        self.board = [0,0,0,0,0,0,0,0,0] # 0 = empty, 1 = player 1, 2 = player 2
 
        
    def Clone(self):
        """ Create a deep clone of this game state.
        """
        st = OXOState()
        st.playerJustMoved = self.playerJustMoved
        st.board = self.board[:]
        return st

    def DoMove(self, move):
        """ Update a state by carrying out the given move.
            Must update playerToMove.
        """
        assert move >= 0 and move <= 8 and move == int(move) and self.board[move] == 0
        self.playerJustMoved = 3 - self.playerJustMoved
        self.board[move] = self.playerJustMoved
        
    def GetMoves(self):
        """ Get all possible moves from this state.
        """
        return [i for i in range(9) if self.board[i] == 0]
    
    def GetResult(self, playerjm):
        """ Get the game result from the viewpoint of playerjm. 
        """
        for (x,y,z) in [(0,1,2),(3,4,5),(6,7,8),(0,3,6),(1,4,7),(2,5,8),(0,4,8),(2,4,6)]:
            if self.board[x] == self.board[y] == self.board[z]:
                if self.board[x] == playerjm:
                    return 1.0
                else:
                    return 0.0
        if self.GetMoves() == []: return 0.5 # draw
        return False # Should not be possible to get here

    def __repr__(self):
        s= ""
        for i in range(9): 
            s += ".XO"[self.board[i]]
            if i % 3 == 2: s += "\n"
        return s
    
#----OC-170220-----
    def getStateData(self):
        myData =np.zeros([0,9], dtype=int)
        for i in range(9):
          myData = np.append(myData, self.board[i])
        myData_str = np.array2string(myData, separator=', ')
        myData_str=myData_str[1:-1] 
        print (myData_str)
        return myData_str 
    
       
class Node:
    """ A node in the game tree. Note wins is always from the viewpoint of playerJustMoved.
        Crashes if state not specified.
    """
    def __init__(self, move = None, parent = None, state = None):
        self.move = move # the move that got us to this node - "None" for the root node
        self.parentNode = parent # "None" for the root node
        self.childNodes = []
        self.wins = 0
        self.visits = 0
        self.untriedMoves = state.GetMoves() # future child nodes
        self.playerJustMoved = state.playerJustMoved # the only part of the state that the Node needs later
        
    def UCTSelectChild(self):
        """ Use the UCB1 formula to select a child node. Often a constant UCTK is applied so we have
            lambda c: c.wins/c.visits + UCTK * sqrt(2*log(self.visits)/c.visits to vary the amount of
            exploration versus exploitation.
        """
        s = sorted(self.childNodes, key = lambda c: c.wins/c.visits + sqrt(2*log(self.visits)/c.visits))[-1]
        return s
    
    def AddChild(self, m, s):
        """ Remove m from untriedMoves and add a new child node for this move.
            Return the added child node
        """
        n = Node(move = m, parent = self, state = s)
        self.untriedMoves.remove(m)
        self.childNodes.append(n)
        return n
    
    def Update(self, result):
        """ Update this node - one additional visit and result additional wins. result must be from the viewpoint of playerJustmoved.
        """
        self.visits += 1
        self.wins += result

    def __repr__(self):
        return "[M:" + str(self.move) + " W/V:" + str(self.wins) + "/" + str(self.visits) + " U:" + str(self.untriedMoves) + "]"

    def TreeToString(self, indent):
        s = self.IndentString(indent) + str(self)
        for c in self.childNodes:
             s += c.TreeToString(indent+1)
        return s

    def IndentString(self,indent):
        s = "\n"
        for i in range (1,indent+1):
            s += "| "
        return s

    def ChildrenToString(self):
        s = ""
        for c in self.childNodes:
             s += str(c) + "\n"
        return s


def UCT(rootstate, itermax, verbose = False, learned_classifier=False):
    """ Conduct a UCT search for itermax iterations starting from rootstate.
        Return the best move from the rootstate.
        Assumes 2 alternating players (player 1 starts), with game results in the range [0.0, 1.0]."""

    rootnode = Node(state = rootstate)

    for i in range(itermax):
        node = rootnode
        state = rootstate.Clone()

        # Select
        while node.untriedMoves == [] and node.childNodes != []: # node is fully expanded and non-terminal
            node = node.UCTSelectChild()
            state.DoMove(node.move)

        # Expand
        if node.untriedMoves != []:  # if we can expand (i.e. state/node is non-terminal)
            m = random.choice(node.untriedMoves) 
            state.DoMove(m)
            node = node.AddChild(m, state)  # add child and descend tree

        # Rollout - this can often be made orders of magnitude quicker using a state.GetRandomMove() function
        while state.GetMoves() != []: # while state is non-terminal
            if learned_classifier: state.DoMove(random.choice(state.GetMoves()))
            else: state.DoMove(random.choice(state.GetMoves()))

        # Backpropagate
        while node != None: # backpropagate from the expanded node and work back to the root node
            node.Update(state.GetResult(node.playerJustMoved)) # state is terminal. Update node with result from POV of node.playerJustMoved
            node = node.parentNode

    # Output some information about the tree - can be omitted
    if verbose: print(rootnode.TreeToString(0))
    else: print(rootnode.ChildrenToString())

    return sorted(rootnode.childNodes, key = lambda c: c.visits)[-1].move # return the move that was most visited

In [0]:
def UCTPlayGame(numberOfGame,df,df2):
    """ Play a sample game between two UCT players where each player gets a different number 
        of UCT iterations (= simulations = tree nodes).
    """
    state = OXOState() # uncomment to play OXO
    #------OC--100420-------------------------------------------
    data_writer=csv.writer(df, delimiter='\t',lineterminator='\n',)
    data_writer2=csv.writer(df2, delimiter='\t',lineterminator='\n',)
    if numberOfGame==0:     
       data_writer.writerow(['game_number, player_just_moved, cell0, cell1, cell2, cell3, cell4, cell5, cell6, cell7, cell8, best_move']) 
       data_writer2.writerow(['game_number, player_won'])
    #-----------------------------------------------------------    
    while state.GetMoves() != []:
        print(str(state))
        if state.playerJustMoved == 1:
            m = UCT(rootstate=state, itermax=1000, verbose=False, learned_classifier=False)  # play with values for itermax and verbose = True
        else:
            m = UCT(rootstate=state, itermax=100, verbose=False, learned_classifier=False) #OC changing intermax to 1000 here results in player 1 winning all the time
        print("Best Move: " + str(m) + "\n")
        #-----OC-170220-----
        data_writer.writerow([numberOfGame]+[',']+[state.playerJustMoved]+[',']+[state.getStateData()]+[',']+[m]) 
        #-------------------
        state.DoMove(m)
        if state.GetResult(state.playerJustMoved) != False:
            print(str(state))
            break  
    if state.GetResult(state.playerJustMoved) == 1.0:
        print("Player " + str(state.playerJustMoved) + " wins!")
        #------OC--100420--to output number of the game and player who won---
        data_writer2.writerow([numberOfGame]+[',']+[state.playerJustMoved]) 
    elif state.GetResult(state.playerJustMoved) == 0.0:
        print("Player " + str(3 - state.playerJustMoved) + " wins!")
        #------OC--100420--to output number of the game and player who won---
        data_writer2.writerow([numberOfGame]+[',']+[3-state.playerJustMoved]) 
    else:
        print("Nobody wins!")


In [4]:
if __name__ == "__main__":
    """ Play a single game to the end using UCT for both players.

    """
    #------OC--100420--iterating over a number of games variable and collecting data for each game---
    with open('data_file.csv',mode='w') as data_file:
       with open('data_filegames.csv',mode='w') as data_file2: 
         for gameNum in range(100):
           UCTPlayGame(gameNum,data_file,data_file2 ) 
    #---train model and save the model---------------
    #---play the game accessing the model and getting he predicted move from the model--


Streaming output truncated to the last 5000 lines.
[M:7 W/V:7.0/50 U:[]]
[M:5 W/V:10.0/57 U:[]]
[M:2 W/V:458.5/841 U:[]]

Best Move: 2

1, 1, 0, 2, 2, 0, 1, 0, 0
XXO
OO.
X..

[M:5 W/V:32.0/64 U:[]]
[M:8 W/V:3.0/19 U:[]]
[M:7 W/V:2.0/17 U:[]]

Best Move: 5

1, 1, 2, 2, 2, 0, 1, 0, 0
XXO
OOX
X..

[M:7 W/V:250.0/500 U:[]]
[M:8 W/V:250.0/500 U:[]]

Best Move: 8

1, 1, 2, 2, 2, 1, 1, 0, 0
XXO
OOX
X.O

[M:7 W/V:50.0/100 U:[]]

Best Move: 7

1, 1, 2, 2, 2, 1, 1, 0, 2
XXO
OOX
XXO

Nobody wins!
...
...
...

[M:4 W/V:6.0/10 U:[]]
[M:7 W/V:2.0/6 U:[0, 3, 6]]
[M:0 W/V:12.0/15 U:[]]
[M:6 W/V:9.0/13 U:[]]
[M:8 W/V:8.5/13 U:[]]
[M:2 W/V:8.5/13 U:[]]
[M:5 W/V:6.0/10 U:[]]
[M:1 W/V:9.0/13 U:[]]
[M:3 W/V:2.5/7 U:[4, 5]]

Best Move: 0

0, 0, 0, 0, 0, 0, 0, 0, 0
X..
...
...

[M:3 W/V:28.0/124 U:[]]
[M:6 W/V:29.5/128 U:[]]
[M:7 W/V:10.0/76 U:[]]
[M:4 W/V:101.5/294 U:[]]
[M:2 W/V:27.0/121 U:[]]
[M:8 W/V:10.5/77 U:[]]
[M:1 W/V:22.5/110 U:[]]
[M:5 W/V:8.0/70 U:[]]

Best Move: 4

1, 0, 0, 0, 0, 0, 0, 0, 0
X..


In [0]:
df_states_and_moves = pd.read_csv("data_file.csv")
df_won_games = pd.read_csv("data_filegames.csv")

In [0]:
df=pd.merge(df_won_games, df_states_and_moves)

In [7]:
display(df)

,game_number,player_won,player_just_moved,cell0,cell1,cell2,cell3,cell4,cell5,cell6,cell7,cell8,best_move
0,5,1,2,0,0,0,0,0,0,0,0,0,3
1,5,1,1,0,0,0,1,0,0,0,0,0,4
2,5,1,2,0,0,0,1,2,0,0,0,0,8
3,5,1,1,0,0,0,1,2,0,0,0,1,1
4,5,1,2,0,2,0,1,2,0,0,0,1,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,97,1,2,0,1,0,0,2,0,0,0,0,2
88,97,1,1,0,1,1,0,2,0,0,0,0,0
89,97,1,2,2,1,1,0,2,0,0,0,0,8
90,97,1,1,2,1,1,0,2,0,0,0,1,3


In [0]:
df.to_csv('data_merged.csv')

In [0]:
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier

In [0]:
dataset=np.loadtxt('data_merged.csv',delimiter=',',skiprows=1, dtype=int)
#loading input data ('player just moved' and state of the cells ) columns into an array
input_data=np.array(dataset[:,3:13], dtype=float)

In [11]:
input_data.shape

(92, 10)

In [12]:
input_data.size
input_data.shape[0]

92

In [13]:
#checking the output: first column has to show the player who moved and other columns show the state of the board
x=input_data[0:20,:]
x

array([[2., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [2., 0., 0., 0., 1., 2., 0., 0., 0., 0.],
       [1., 0., 0., 0., 1., 2., 0., 0., 0., 1.],
       [2., 0., 2., 0., 1., 2., 0., 0., 0., 1.],
       [1., 0., 2., 0., 1., 2., 0., 0., 1., 1.],
       [2., 0., 2., 2., 1., 2., 0., 0., 1., 1.],
       [2., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [2., 0., 0., 1., 0., 2., 0., 0., 0., 0.],
       [1., 0., 1., 1., 0., 2., 0., 0., 0., 0.],
       [2., 2., 1., 1., 0., 2., 0., 0., 0., 0.],
       [1., 2., 1., 1., 0., 2., 0., 0., 0., 1.],
       [2., 2., 1., 1., 2., 2., 0., 0., 0., 1.],
       [2., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [2., 0., 1., 0., 0., 2., 0., 0., 0., 0.],
       [1., 0., 1., 1., 0., 2., 0., 0., 0., 0.],
       [2., 2., 1., 1., 0., 2., 0., 0., 0., 0.],
       [1., 2., 1., 1., 0., 2., 0., 0., 0., 1.]])

In [0]:
#loading results data into an array (next best move)
result_data=np.array(dataset[:,[-1]])

In [15]:
result_data.shape

(92, 1)

In [16]:
#loading results data into an array 
result_data=np.array(dataset[:,[-1]]).reshape(result_data.shape[0])
print(result_data)

[3 4 8 1 7 2 6 2 4 1 0 8 3 5 1 4 2 0 8 3 5 7 0 6 4 8 7 0 6 4 8 1 4 8 3 6 5
 0 4 2 1 5 7 2 0 5 4 8 6 4 3 0 8 1 7 0 4 6 3 7 5 7 0 3 4 8 2 1 6 8 4 5 1 7
 2 0 6 5 0 1 4 8 6 3 2 1 4 2 0 8 3 5]


In [17]:
result_data.shape

(92,)

In [0]:
#splitting data into the training and testing sets
from sklearn.model_selection import train_test_split
x_train,x_test, y_train, y_test=train_test_split(input_data,result_data,test_size=0.2)

In [0]:
from sklearn import metrics
modelDT=DecisionTreeClassifier(criterion="entropy", max_depth=10)
modelDT.fit(x_train,y_train)
y_pred=modelDT.predict(x_test)
score=metrics.accuracy_score(y_test,y_pred)

In [20]:
score

0.42105263157894735

In [0]:
from sklearn.svm import SVC


In [0]:
  modelSV2=SVC(kernel='poly', degree=2)
  modelSV2.fit(x_train,y_train)
  y_pred=modelSV2.predict(x_test)
  score=metrics.accuracy_score(y_test,y_pred)

In [23]:
score

0.3157894736842105

In [0]:
import pickle
# save the model 
filename = 'saved_model.sav'
pickle.dump(modelDT, open(filename, 'wb'))

In [25]:
# load the model 
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(x_test,y_test)
print(result)

0.42105263157894735
